<a href="https://colab.research.google.com/github/HarisudhanVL/Natural-Language-Processing/blob/main/English_to_Hindi_Translation_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Required Liraries.**

In [ ]:
import re
import os
import json
import yaml
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import preprocessing , utils
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers , activations , models , preprocessing

## **Loading Dataset.**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hin.txt to hin.txt


In [ ]:
docs=open('/content/hin.txt',encoding='utf-8').read().split("\n")
c=docs[0].strip().split("\t")

In [ ]:
len(docs)

2003

## **Chunks of Data.**

In [ ]:
questions_for_token = list()
answers_for_token = list()
c=1
for con in docs:
    if(c==2001):
        pass
    else:
        print(con)
        con=con.strip().split("\t")
        questions_for_token.append(con[0])
        answers_for_token.append(con[1])
        c+=1

embed_size=100 #define the vector size based on word your embedding
max_features=6000 #to restrict your number of unique words
maxlen=100

Help!	बचाओ!
Jump.	उछलो.
Jump.	कूदो.
Jump.	छलांग.
Hello!	नमस्ते।
Hello!	नमस्कार।
Cheers!	वाह-वाह!
Cheers!	चियर्स!
Got it?	समझे कि नहीं?
I'm OK.	मैं ठीक हूँ।
Awesome!	बहुत बढ़िया!
Come in.	अंदर आ जाओ।
Get out!	बाहर निकल जाओ!
Go away!	चले जाओ!
Goodbye!	ख़ुदा हाफ़िज़।
Perfect!	उत्तम!
Perfect!	सही!
Welcome.	आपका स्वागत है।
Welcome.	स्वागतम्।
Have fun.	मज़े करना।
Have fun.	मौज करना।
Have fun.	मज़े करो।
I forgot.	मैं भूल गया।
I forgot.	मैं भूल गई।
I'll pay.	मैं पैसे दूंगा।
I'm fine.	मैं ठीक हूँ।
I'm full.	मेरा पेट भर गया है।
Let's go!	चलो चलें!
Answer me.	मुझे जवाब दो।
Birds fly.	पंछी उड़ते हैं।
Excuse me.	माफ़ कीजिए।
Fantastic!	बहुत ख़ूब!
I fear so.	खेद की बात है, लेकिन वैसा ही है।
I laughed.	मैं हँसा।
I'm bored.	मैं बोर हो रहा हूँ।
I'm broke.	मेरा दीवालिया हो चुका है।
I'm tired.	मैं थक गया हूँ।
It's cold.	ठंड हो रही है।
Who knows?	कौन जाने?
Who knows?	किसको पता है?
Who knows?	किसे पता है?
Who knows?	किसे मालूम है?
Wonderful!	अद्भुत
Birds sing.	पंछी गाते हैं।
Come on in.	अंदर आ जाओ।
Definite

In [ ]:
questions_for_token

['Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Got it?',
 "I'm OK.",
 'Awesome!',
 'Come in.',
 'Get out!',
 'Go away!',
 'Goodbye!',
 'Perfect!',
 'Perfect!',
 'Welcome.',
 'Welcome.',
 'Have fun.',
 'Have fun.',
 'Have fun.',
 'I forgot.',
 'I forgot.',
 "I'll pay.",
 "I'm fine.",
 "I'm full.",
 "Let's go!",
 'Answer me.',
 'Birds fly.',
 'Excuse me.',
 'Fantastic!',
 'I fear so.',
 'I laughed.',
 "I'm bored.",
 "I'm broke.",
 "I'm tired.",
 "It's cold.",
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Wonderful!',
 'Birds sing.',
 'Come on in.',
 'Definitely!',
 "Don't move.",
 'Fire burns.',
 'Follow him.',
 'I am tired.',
 'I can swim.',
 'I can swim.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I will try.',
 "I'm coming.",
 "I'm hungry!",
 "I'm hungry!",
 'Let him in.',
 'Let him in.',
 'Let me out!',
 'Once again.',
 'Please sit.',
 "What's new?",
 "What's new?",
 "Who's that?",
 "Don'

In [ ]:
answers_for_token

['बचाओ!',
 'उछलो.',
 'कूदो.',
 'छलांग.',
 'नमस्ते।',
 'नमस्कार।',
 'वाह-वाह!',
 'चियर्स!',
 'समझे कि नहीं?',
 'मैं ठीक हूँ।',
 'बहुत बढ़िया!',
 'अंदर आ जाओ।',
 'बाहर निकल जाओ!',
 'चले जाओ!',
 'ख़ुदा हाफ़िज़।',
 'उत्तम!',
 'सही!',
 'आपका स्वागत है।',
 'स्वागतम्।',
 'मज़े करना।',
 'मौज करना।',
 'मज़े करो।',
 'मैं भूल गया।',
 'मैं भूल गई।',
 'मैं पैसे दूंगा।',
 'मैं ठीक हूँ।',
 'मेरा पेट भर गया है।',
 'चलो चलें!',
 'मुझे जवाब दो।',
 'पंछी उड़ते हैं।',
 'माफ़ कीजिए।',
 'बहुत ख़ूब!',
 'खेद की बात है, लेकिन वैसा ही है।',
 'मैं हँसा।',
 'मैं बोर हो रहा हूँ।',
 'मेरा दीवालिया हो चुका है।',
 'मैं थक गया हूँ।',
 'ठंड हो रही है।',
 'कौन जाने?',
 'किसको पता है?',
 'किसे पता है?',
 'किसे मालूम है?',
 'अद्भुत',
 'पंछी गाते हैं।',
 'अंदर आ जाओ।',
 'निश्चित ही',
 'हिलो मत।',
 'आग जलाती है।',
 'उसका पीछा करो।',
 'मैं थक गया हूँ।',
 'मुझे तैरना आता है।',
 'मैं तैर सकता हूँ।',
 'मैं तुमसे प्यार करती हूँ।',
 'मैं तुमसे प्यार करता हूँ।',
 'मैं आपसे प्यार करता हूँ।',
 'मैं आपसे प्यार करती हूँ।',
 'मुझे तुमस

## **Text Preprocessing.**

In [ ]:
def processTweet(chat):
    chat = chat.lower()
    chat = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',chat)
    chat = re.sub('@[^\s]+','',chat)
    chat = re.sub('[\s]+', ' ', chat)
    chat = re.sub(r'#([^\s]+)', r'\1', chat)
    chat = re.sub(r'[\.!:\?\-\'\"\\/]', r'', chat)
    chat = chat.strip('\'"')
    return chat

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    return pattern.sub(r"\1", s)

def getFeatureVector(chat):
    chat=processTweet(chat)
    featureVector = []
    #split tweet into words
    words = chat.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(val is None):
            continue
        else:
            featureVector.append(w.lower())
    return " ".join(list(featureVector))

## Word Embedding - GloVe Model.

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-04-20 02:28:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-20 02:28:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-20 02:28:11--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# !unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
def emb_mat(nb_words):
    EMBEDDING_FILE="/content/drive/MyDrive/GloVe/glove.6B.100d.txt"
    def get_coefs(word,*arr):
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    emb_mean,emb_std

    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words+1, embed_size))
    for word, i in word_index.items():
        if (i >= max_features) or i==nb_words:
            continue
        embedding_vector = embeddings_index.get(word) #here we will get embedding for each word from GloVe
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

## **Data Tokenization.**

In [ ]:
def tokenized_data(questions,answers,VOCAB_SIZE,tokenizer):
    # encoder_input_data
    import numpy as np
    tokenized_questions = tokenizer.texts_to_sequences( questions )
    maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
    padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen , padding='post' )
    encoder_input_data = np.array( padded_questions )
    #print( encoder_input_data.shape , maxlen_questions )

    # decoder_input_data
    tokenized_answers = tokenizer.texts_to_sequences( answers )
    maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen , padding='post' )
    decoder_input_data = np.array( padded_answers )
    #print( decoder_input_data.shape , maxlen_answers )

    # decoder_output_data
    tokenized_answers = tokenizer.texts_to_sequences( answers )
    for i in range(len(tokenized_answers)) :
        tokenized_answers[i] = tokenized_answers[i][1:] # remove <start> take rest
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen , padding='post' )
    onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE)
    decoder_output_data = np.array( onehot_answers )
    #print( decoder_output_data.shape )

    return [encoder_input_data,decoder_input_data,decoder_output_data,maxlen_answers]

## **Data Preparation.**

In [ ]:
def prepare_data(questions,answers):
    answers=pd.DataFrame(answers, columns=["Ans"])
    questions=pd.DataFrame(questions, columns=["Question"])
    questions["TokQues"]=questions["Question"].apply(getFeatureVector)

    answers=np.array(answers["Ans"])
    questions=np.array(questions["TokQues"])

    answers_with_tags = list()
    for i in range( len( answers ) ):
        if type( answers[i] ) == str:
            answers_with_tags.append( answers[i] )
        else:
            print(questions[i])
            print(answers[i])
            print(type(answers[i]))
            questions.pop(i)

    answers = list()
    for i in range( len( answers_with_tags ) ) :
        answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

    tokenizer = preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(questions+answers)

    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))

    #embedding_matrix=emb_mat(nb_words)[0]
    #emb_vec=emb_mat(nb_words)[1]

    VOCAB_SIZE = len( tokenizer.word_index )+1

    tok_out=tokenized_data(questions,answers,VOCAB_SIZE,tokenizer)
    encoder_input_data=tok_out[0]
    decoder_input_data=tok_out[1]
    decoder_output_data=tok_out[2]
    maxlen_answers=tok_out[3]

    return [encoder_input_data,decoder_input_data,decoder_output_data,maxlen_answers,nb_words,word_index,tokenizer]

## **Model Train.**

In [ ]:
Prepared_data=prepare_data(questions_for_token,answers_for_token)
encoder_input_data=Prepared_data[0]
decoder_input_data=Prepared_data[1]
decoder_output_data=Prepared_data[2]
maxlen_answers=Prepared_data[3]
nb_words=Prepared_data[4]
word_index=Prepared_data[5]
tokenizer=Prepared_data[6]
embedding_matrix=emb_mat(nb_words)

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True, weights=[embedding_matrix]) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( nb_words+1, embed_size , mask_zero=True,weights=[embedding_matrix]) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )

decoder_dense = tf.keras.layers.Dense( nb_words+1 , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )

model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=100)

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


Epoch 1/100
200/200 [==============================] - 23s 54ms/step - loss: 5.2655 - accuracy: 0.2589
Epoch 2/100
200/200 [==============================] - 6s 31ms/step - loss: 4.6777 - accuracy: 0.2885
Epoch 3/100
200/200 [==============================] - 5s 26ms/step - loss: 4.4739 - accuracy: 0.3089
Epoch 4/100
200/200 [==============================] - 7s 33ms/step - loss: 4.2887 - accuracy: 0.3335
Epoch 5/100
200/200 [==============================] - 5s 27ms/step - loss: 4.1276 - accuracy: 0.3541
Epoch 6/100
200/200 [==============================] - 6s 28ms/step - loss: 3.9793 - accuracy: 0.3711
Epoch 7/100
200/200 [==============================] - 6s 30ms/step - loss: 3.8249 - accuracy: 0.3889
Epoch 8/100
200/200 [==============================] - 5s 27ms/step - loss: 3.6774 - accuracy: 0.4038
Epoch 9/100
200/200 [==============================] - 6s 32ms/step - loss: 3.5393 - accuracy: 0.4203
Epoch 10/100
200/200 [==============================] - 5s 26ms/step - loss: 3.39

## **Inference.**

In [ ]:
def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

In [ ]:
# for _ in range(20):
#     print("Hiii I am HinBot, I will help you to perform translation from English to Hindi.")
#     try:
#         states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
#         empty_target_seq = np.zeros( ( 1 , 1 ) )
#         empty_target_seq[0, 0] = tokenizer.word_index['start']
#         stop_condition = False
#         decoded_translation = ''
#         while not stop_condition :
#             dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
#             sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
#             sampled_word = None
#             for word , index in tokenizer.word_index.items() :
#                 if sampled_word_index == index :
#                     decoded_translation += ' {}'.format( word )
#                     sampled_word = word

#             if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
#                 stop_condition = True

#             empty_target_seq = np.zeros( ( 1 , 1 ) )
#             empty_target_seq[ 0 , 0 ] = sampled_word_index
#             states_values = [ h , c ]

#         print( " ".join(decoded_translation.strip().split(" ")[:-1]) )
#     except:
#         print("Sorry, I don't understand. Please try again.")

print("Chatbot: Hi, I'm a language model chatbot. Please enter the text:")

while True:
    try:
        user_input = input('You: ')
        states_values = enc_model.predict(str_to_tokens(user_input))

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = tokenizer.word_index['start']
        stop_condition = False
        decoded_translation = ''

        while not stop_condition:
            dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)
            sampled_word_index = np.argmax(dec_outputs[0, -1, :])
            sampled_word = None

            for word, index in tokenizer.word_index.items():
                if sampled_word_index == index:
                    decoded_translation += ' {}'.format(word)
                    sampled_word = word

            if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True

            empty_target_seq = np.zeros((1, 1))
            empty_target_seq[0, 0] = sampled_word_index
            states_values = [h, c]

        print("Chatbot:", " ".join(decoded_translation.strip().split(" ")[:-1]))

    except:
        print("Chatbot: Sorry, I don't understand. Can you please rephrase your question?")

    if user_input.lower() == 'bye':
        print("Chatbot: Goodbye!")
        break

Chatbot: Hi, I'm a language model chatbot. Please enter the text:
You: hii
Chatbot: Sorry, I don't understand. Can you please rephrase your question?
You: Hello
1/1 [==============================] - 0s 46ms/step
Chatbot: नमस्कार।
You: harisudhan
Chatbot: Sorry, I don't understand. Can you please rephrase your question?
You: It's very hot here
Chatbot: Sorry, I don't understand. Can you please rephrase your question?
You: It is very hot here
1/1 [==============================] - 0s 33ms/step
Chatbot: यहाँ बहुत गरम है ना
You: Bye
Chatbot: Sorry, I don't understand. Can you please rephrase your question?
Chatbot: Goodbye!
